# Chapter 12: Markov chain Monte Carlo

## Metropolis-Hastings

Here's how to implement the Metropolis-Hastings algorithm for Example 12.1.8, the Normal-Normal model. First, we choose our observed value of $Y$ and decide on values for the constants $\sigma$, $\mu$ and $\tau$:

In [1]:
y <- 3
sigma <- 1
mu <- 0
tau <- 2

We also need to choose the standard deviation of the proposals for step $1$ of the algorithm, as explained in Example 12.1.8; for this problem, we let $d= 1$. We set the number of iterations to run, and we allocate a vector `theta` of length $10^4$ which we will fill with our simulated draws:

In [2]:
d <- 1
niter <- 10^4
theta <- rep(0,niter)

Now for the main loop. We initialize $\theta$ to the observed value $y$, then run the algorithm described in Example 12.1.8:

In [3]:
theta[1] <- y
for (i in 2:niter){
    theta.p <- theta[i-1] + rnorm(1,0,d)
    r <- dnorm(y,theta.p,sigma) * dnorm(theta.p,mu,tau) /
        (dnorm(y,theta[i-1],sigma) * dnorm(theta[i-1],mu,tau))
    flip <- rbinom(1,1,min(r,1))
    theta[i] <- if(flip==1) theta.p else theta[i-1]
}

Let's step through each line inside the loop. The proposed value of $\theta$ is `theta.p`, which equals the previous value of $\theta$ plus a Normal random variable with mean $0$ and standard deviation `d` (recall that `rnorm` takes the standard deviation and not the variance as input). The ratio `r` is
$$ \frac{f_{\theta|Y}(x'|y)}{f_{\theta|Y}(x|y)} = \frac{e^{- \frac{1}{2 \sigma^2}(y-x')^2} e^{-\frac{1}{2\tau^2}(x' - m)^2}}{e^{- \frac{1}{2 \sigma^2}(y-x)^2} e^{-\frac{1}{2\tau^2}(x - m)^2}},$$
where `theta.p` is playing the role of $x'$ and `theta[i-1]` is playing the role of $x$. The coin flip to determine whether to accept or reject the proposal is `flip`, which is a coin flip with probability `min(r,1)` of Heads (encoding Heads as $1$ and Tails as $0$). Finally, we set `theta[i]` equal to the proposed value if the coin flip lands Heads, and keep it at the previous value otherwise.

The vector `theta` now contains all of our simulation draws. We typically discard some of the initial draws to give the chain some time to approach the stationary distribution. The following line of code discards the first half of the draws:

In [4]:
theta <- theta[-(1:(niter/2))]

To see what the remaining draws look like, we can create a histogram using `hist(theta)`. We can also compute summary statistics such as `mean(theta)` and `var(theta)`, which give us the sample mean and sample variance.

## Gibbs

Now let's implement Gibbs sampling for Example 12.2.6, the chicken-egg story with unknown hatching probability and invisible unhatched eggs. The first step is to decide on our observed value of $X$, as well as the constants $\lambda,a,b$:

In [5]:
x <- 7
lambda <- 10
a <- 1
b <- 1

Next we decide how many iterations to run, and we allocate space for our results, creating two vectors `p` and `N` of length $10^4$ which we will fill with our simulated draws:

In [6]:
niter <- 10^4
p <- rep(0,niter)
N <- rep(0,niter)

Finally, we're ready to run the Gibbs sampler. We initialize $p$ and $N$ to the values $0.5$ and $2x$, respectively, and then we run the algorithm as explained in Example 12.2.6:

In [7]:
p[1] <- 0.5
N[1] <- 2*x
for (i in 2:niter){
    p[i] <- rbeta(1,x+a,N[i-1]-x+b)
    N[i] <- x + rpois(1,lambda*(1-p[i-1]))
}

Again, we discard the initial draws:

In [8]:
p <- p[-(1:(niter/2))]
N <- N[-(1:(niter/2))]

To see what the remaining draws look like, we can make histograms using `hist(p)` and `hist(N)`, which is how we created Figure 12.5. We can also compute summary statistics such as `mean(p)` or `median(p)`.